In [ ]:
import numpy as np
import itertools
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, Lock, Manager
import os
import time
import uuid
from symmetry_breaking.models.NLP_neutralization_1D import NodalLeftyNeutralization1D
from symmetry_breaking.models.sweep import run_simulation_1D, make_1d_grid
from symmetry_breaking.models.trackers import NodalROITracker

In [ ]:
from pde import ScalarField, FieldCollection, PDEBase, CartesianGrid

def make_1d_grid(length=3000, dx=10, periodic=True):
    N = int(length / dx)
    grid = CartesianGrid([[0, length]], shape=(N,), periodic=periodic)
    return grid

In [ ]:
# hyperparams
dx = 5
L = 3500
T = 10 * 60 * 60
dt = 0.5 * dx ** 2 / 60 / 1.25 # Stability condition for diffusion

param_dicts= [{
    "D0_N":60.0, 
    "D0_L":60.0,
    "alpha_N":0,
    "alpha_L":0,
    "sigma_L": 1, #np.logspace(-3, -1, 10),
    "sigma_N": 10,  #np.logspace(-1, 1, 10),
    # "lambda_": .01,
    # "K_R": 100, #np.logspace(1, 3, 10),
    "K_I":100,
    "K_A": 2154, #np.logspace(1.81, 3.81, 10),
    "K_NL": 77,#np.logspace(1.81, 3.81, 10),
    "K_rho": 2154,
    "K_R":46
    # "alpha_N":5,
    # "alpha_L": 0.0001
}]

static_params = {
              # "mu_N": 1.11e-3,
              "mu_L": 0.61e-3,
              # "lambda_": 0.1,    # Lefty inhibition rate
              "n": 2,
              "m": 1,
              "p": 2,
              "q": 2, # cell density nodal-dependence
              "tau_rho":3600, # pretty raoid cellular relacation
              "N_amp":5000, 
              "N_sigma":50.0,
              "rho_value":0.1
                }

# Simulation config (you can also pass this from CLI)
sim_config = {
                "dx": dx,
                "L": L,
                "T": T,
                "dt": dt,
                "model_class": NodalLeftyNeutralization1D,
                "tracker_class": NodalROITracker,
                "interval": 1000,
            }

args = [(p | static_params, sim_config, "") for p in param_dicts]

# args

In [ ]:
from pde import ProgressTracker, PlotTracker

param_dict, sim_config, output_dir = args[0]

# Unpack sim config
dx = sim_config["dx"]
L = sim_config["L"]
T = sim_config["T"]
dt = sim_config["dt"]
model_class = sim_config["model_class"]
tracker_class = sim_config["tracker_class"]
interval = sim_config.get("interval", 1000)

# --- Setup grid and model ---
grid = make_1d_grid(length=L, dx=dx)
model = model_class(**param_dict)
state = model.get_state(grid)

# --- Build a tracker collection ---
# Progress bar updates every 'interval' steps
progress = ProgressTracker(interval=interval)

# Live plots (you can adjust scale, cmap, etc.)
plots = PlotTracker(interval=interval, plot_args={"figsize": (6, 4)}, 
                    show=True, tight_layout=True)

# If you want both at once, wrap them in a list
trackers = [progress, plots, tracker_class(grid, interval=interval)]

# --- Run simulation ---
state = model.solve(state, t_range=T, dt=dt, tracker=trackers)

# --- Collect results from your custom tracker ---
result = {
    **param_dict,
    **trackers[-1].get_metrics(),  # last tracker is your custom one
}

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

x_grid = np.linspace(0, L, int(L/dx)) - L/dx/2

fig = go.Figure()
fig.add_traces(go.Scatter(
    x=x_grid, 
    y=result["profile_Activator"] / result["K_A"], 
    mode="lines", 
    line=dict(width=3, color="#8da0cb"),  # Set2 blue
    fill="tozeroy",
    fillcolor="rgba(141, 160, 203, 0.3)",  # semi-transparent blue
    name="Nodal"
))
fig.add_traces(go.Scatter(
    x=x_grid, 
    y=result["profile_Repressor"] / result["K_I"], 
    mode="lines", 
    line=dict(width=3, color="#fc8d62"),  # Set2 red
    fill="tozeroy",
    fillcolor="rgba(252, 141, 98, 0.3)",  # semi-transparent red
    name="Lefty"
))
fig.show()



In [ ]:
result